In [ ]:
import torch
import torch.nn as nn
from scipy.linalg import null_space

Goal: find the probability distribution $p = (p_1, \ldots, p_6)$ which maximizes

$$H(p) = -\sum_i p_i \log p_i$$

subject to the constraints

$$\sum_i p_i = 1, \quad \sum_i i p_i = 4.5$$

Interpretation: $p_i$ represents the probability that a six sided die is rolled and comes up with the pip $i$.
The first constraint ensures that this is a probability distribution, and the second asserts that the average die roll is $4.5$.
The objective function $H$ gives the entropy of the distribution.

Our strategy will be to run gradient descent along the (negative) entropy function, but there is a catch: most of the time gradient descent will pull us out of the space defined by the constraints.
We will fix this by orthogonally projecting back onto the constraint space after reach step.

Express the constraints as the matrix equation $Ax = b$ where:

$$A = \begin{pmatrix} 1 & 1 & 1 & 1 & 1 & 1 \\ 1 & 2 & 3 & 4 & 5 & 6 \end{pmatrix}, \quad b = \begin{pmatrix} 1 \\ 4.5\end{pmatrix}$$

Let $L$ denote the solution space for the equation $Ax = b$; this is a 4-dimensional affine hyperplane in $\mathbb{R}^6$.
Our aim is to construct the map $P_L \colon \mathbb{R}^6 \to \mathbb{R}^6$ which orthogonally projects onto $L$.

We start with the singular value decomposition of $A$:

In [ ]:
A = torch.tensor([[1,1,1,1,1,1], [1,2,3,4,5,6]], dtype=float)
b = torch.tensor([[1], [4.5]], dtype=float)

In [ ]:
U, S, VT = torch.linalg.svd(A)
V = torch.transpose(VT, 0, 1)

In [ ]:
# Quick check that torch.linalg.svd works the way I think it does:
S_matrix = torch.zeros(2, 6, dtype=float)
S_matrix[:, :len(S)] = torch.diag(S)
torch.linalg.multi_dot([U, S_matrix, VT])

The last `len(S)` columns of $V$ form an orthonormal basis for the kernel of $A$:

In [ ]:
K = V[:, len(S):]

The matrix $P = K K^T$ is the orthogonal projector onto the kernel of $A$.
We can check this by verifying that $P^2 = P$ and that $AP = 0$

In [ ]:
P = torch.mm(K, torch.transpose(K, 0, 1))

In [ ]:
torch.mm(P, P) - P

In [ ]:
torch.mm(A, P)

Finally, to construct the affine projector $P_L$ we just need to find any vector $v \in L$ and then write:

$$P_L x = v + P(x - v)$$

(The easiest way to solve generic matrix equations in Pytorch appears to be `torch.linalg.lstsq`, which finds the least squares solution, but any other solution would work fine.)

In [ ]:
v = torch.linalg.lstsq(A, b).solution
v

Let's put all of this together into a Pytorch module:

In [ ]:
class MaxEntropyDice(torch.nn.Module):
    def __init__(self, num_faces, mean_constraint):
        super().__init__()
        self.num_faces = num_faces
        self.mean_constraint = mean_constraint
        self.kernel_projector, self.basepoint = self._compute_constraint_projector()
        self.p = nn.Parameter(nn.functional.normalize(torch.rand(num_faces), p=1, dim=0))
    
    def forward(self):
        entropy = -torch.sum(self.p * torch.log(self.p))
        return entropy
    
    def enforce_constraint(self):
        projected_p = self.basepoint + torch.mm(self.kernel_projector, self.p.reshape(len(self.p), 1) - self.basepoint)
        self.p.copy_(projected_p.flatten())
    
    def _compute_constraint_projector(self):
        probability_coeffs = torch.ones(self.num_faces)
        mean_coeffs = torch.tensor(range(1, self.num_faces+1))
        A = torch.stack((probability_coeffs, mean_coeffs))
        b = torch.tensor([[1], [self.mean_constraint]])
        
        U, S, VT = torch.linalg.svd(A) # Singular value decomposition
        V = torch.transpose(VT, 0, 1)
        K = V[:, len(S):] # Orthonormal basis for kernel of A
        
        P = torch.mm(K, torch.transpose(K, 0, 1)) # Orthogonal projector onto the kernel of A
        v = torch.linalg.lstsq(A, b).solution # A specific solution to Ax = b (the least squares solution)
        return (P, v)

In [ ]:
model = MaxEntropyDice(num_faces=6, mean_constraint=4.5)

Now we return to the task of maximizing entropy using gradient descent, projecting back onto the constraints at each step:

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

for i in range(2000):
    optimizer.zero_grad()
    loss = -model() # Reverse the sign since the optimizer seeks a minimum
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        model.enforce_constraint()

In [ ]:
model.p